<a href="https://colab.research.google.com/github/RajBharti25/Codalab-Sentiment-Ananlysis-/blob/master/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!pwd

/content/gdrive/My Drive/Colab Notebooks/CodaLab/working_directory


In [0]:
cd gdrive/My Drive/Colab Notebooks/CodaLab/working_directory

/content/gdrive/My Drive/Colab Notebooks/CodaLab/working_directory


In [0]:
import numpy as np
import pandas as pd
import re
np.random.seed(0)
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from keras import optimizers
from keras.preprocessing import sequence
from keras.layers import Dense,Input,Dropout,LSTM,Activation,Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)

In [8]:
file=open('/content/gdrive/My Drive/Colab Notebooks/CodaLab/Data/EI-oc/EI-oc-En-train/EI-oc-En-anger-train.txt')
f=file.read()
Tweet=[]
Class=[]
lines=f.split('\n')[1:]
i=0
for line in lines:
  item=line.split('\t')
  if(len(item)<4):
    #print(i)
    i=i+1
    continue
  Tweet.append(item[1])
  Class.append(item[3])
for i in range(len(Class)):
  Class[i]=int(str(Class[i])[0])
Class=np.array(Class)
Tweet=np.array(Tweet)
file.close()
for i in range(5):
  print(Tweet[i])
  print(Class[i])

@xandraaa5 @amayaallyn6 shut up hashtags are cool #offended
2
it makes me so fucking irate jesus. nobody is calling ppl who like hajime abusive stop with the strawmen lmao
3
Lol Adam the Bull with his fake outrage...
1
@THATSSHAWTYLO passed away early this morning in a fast and furious styled car crash as he was leaving an ATL strip club. That's rough stuff
0
@Kristiann1125 lol wow i was gonna say really?! haha have you seen chris or nah? you dont even snap me anymore dude!
1


In [0]:
UNK_value=[-0.12920076, -0.28866628, -0.01224866, -0.05676644, -0.20210965, -0.08389011,
      0.33359843,  0.16045167,  0.03867431,  0.17833012,  0.04696583, -0.00285802,
      0.29099807,  0.04613704, -0.20923874, -0.06613114, -0.06822549,  0.07665912,
      0.3134014,   0.17848536, -0.1225775,  -0.09916984, -0.07495987,  0.06413227,
      0.14441176,  0.60894334,  0.17463093,  0.05335403, -0.01273871,  0.03474107,
      -0.8123879,  -0.04688699,  0.20193407,  0.2031118,  -0.03935686,  0.06967544,
      -0.01553638, -0.03405238, -0.06528071,  0.12250231,  0.13991883, -0.17446303,
      -0.08011883,  0.0849521,  -0.01041659, -0.13705009,  0.20127155,  0.10069408,
       0.00653003,  0.01685157]

In [0]:
def read_glove_vecs(glove_file):
  with open(glove_file,'r') as f:
    words=set()
    words_to_vec_map={}
    for line in f:
      line =line.strip().split()
      curr_word=line[0]
      words.add(curr_word)
      words_to_vec_map[curr_word]=np.array(line[1:], dtype=np.float64)
    i=1
    words_to_index={}
    index_to_words={}
    for w in sorted(words):
      words_to_index[w]=i
      index_to_words[i]=w
      i=i+1
    words=sorted(words)
    words.append('<unk>')
    words_to_index['<unk>']=i
    index_to_words[i] = '<unk>'
    words_to_vec_map['<unk>']=UNK_value

  return words,words_to_index, index_to_words, words_to_vec_map
  

In [0]:
words_set,word_to_index, index_to_word, word_to_vec_map = read_glove_vecs(r'glove.6B.50d.txt')

In [12]:
if('<unk1>' in words_set):
  print('true')
else:
  print('false')


false


In [0]:
def convert_to_one_hot(Y,C):
  Y = np.eye(C)[Y.reshape(-1)]
  return Y

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()


In [0]:
Y_train=to_categorical(Class, num_classes=4)

In [15]:
Y_train.shape

(1701, 4)

In [16]:
word='king'
idx = 289846
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(idx) + "th word in the vocabulary is", index_to_word[idx])

the index of king in the vocabulary is 207802
the 289846th word in the vocabulary is potatos


In [17]:
l=str(Tweet).strip().split()
maxLen = len(max(l, key=len))
maxLen

14

In [0]:
#sentence to indices function so that each word can be embedded with embedding vector
def sentence_to_indices(X,word_to_index,max_len,words_set):
  m=X.shape[0]
  X_indices=np.zeros(shape=(m,max_len))
  for i in range(m):
    sentence=X[i]
    sentence=re.sub(r'@(?i)[a-zA-Z0-9_]+','',sentence)
    sentence=re.sub('[^a-zA-z0-9\s]','',sentence)
    words=sentence.lower().strip().split()
    
    j=0
    for w in words:
      if(w in words_set):
        X_indices[i,j]=word_to_index[w]
        j=j+1
      else:
        X_indices[i,j]=word_to_index['<unk>']
        j=j+1
  return X_indices

In [19]:
X_train=sentence_to_indices(Tweet,word_to_index,40,words_set)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Flags not at the start of the expression '@(?i)[a-zA-Z0-9_]+'
  


In [35]:
file2=open('/content/gdrive/My Drive/Colab Notebooks/CodaLab/Data/EI-oc/2018-EI-oc-En-dev/2018-EI-oc-En-anger-dev.txt')
f=file2.read()
Tweet1=[]
Class1=[]
lines=f.split('\n')[1:]
i=0
for line in lines:
  item=line.split('\t')
  if(len(item)<4):
    #print(i)
    i=i+1
    continue
  Tweet1.append(item[1])
  Class1.append(item[3])
for i in range(len(Class1)):
  Class1[i]=int(str(Class1[i])[0])
Class1=np.array(Class1)
Tweet1=np.array(Tweet1)
file.close()
for i in range(5):
  print(Tweet1[i])
  print(Class1[i])

#xx_DEV,y_DEV
X_dev=sentence_to_indices(Tweet1,word_to_index,40,words_set)
Y_dev=to_categorical(Class1, num_classes=4)

'we need to do something. something must be done!!!!!'\n\nyour anxiety is amusing. nothing will be done. despair.
1
@Chan_lfc10 @paul_rule @Nuttall1878 @DeadlineDayLive @Everton We'd be fuming if the hijacked our £8m move for a relegated full back 😡
2
Caleb had a nightmare about zombies. I had a dream about freedom.......
0
#CNN really needs to get out of the #Propaganda Business.. 30 seconds on USN fallen Soldiers tragedy. Right back at spewing #hatred #POTUS
2
#dmme #kikme  #sext #horny  #ass #bbw  #naughty  #pussy #kik   #nudes  only  girls  😩 horny  #snap    jacobgigs
0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Flags not at the start of the expression '@(?i)[a-zA-Z0-9_]+'
  


In [0]:
file3=open('/content/gdrive/My Drive/Colab Notebooks/CodaLab/Data/EI-oc/test-gold/2018-EI-oc-En-anger-test-gold.txt')
f=file3.read()
Tweet2=[]
Class2=[]
lines=f.split('\n')[1:]
i=0
for line in lines:
  item=line.split('\t')
  if(len(item)<4):
    #print(i)
    i=i+1
    continue
  Tweet2.append(item[1])
  Class2.append(item[3])
for i in range(len(Class2)):
  Class2[i]=int(str(Class2[i])[0])
Class2=np.array(Class2)
Tweet2=np.array(Tweet2)
file.close()
for i in range(5):
  print(Tweet2[i])
  print(Class2[i])
print('size of test data:',len(Tweet2))
#xx_DEV,y_DEV
X_test=sentence_to_indices(Tweet2,word_to_index,40,words_set)
Y_test=to_categorical(Class2, num_classes=4)

In [0]:
X_train1=np.concatenate((X_train,X_test),axis=0)
Y_train1=np.concatenate((Y_train,Y_test),axis=0)

In [0]:
#  pretrained_embedding_layer

def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    vocab_len = len(word_to_index) + 1                
    emb_dim = word_to_vec_map["cucumber"].shape[0]  
    emb_matrix = np.zeros(shape=(vocab_len,emb_dim))
  
    for word, idx in word_to_index.items():
        emb_matrix[idx, :] = word_to_vec_map[word]

    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    embedding_layer.build((None,)) 
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [21]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])










weights[0][1][3] = -0.3403


In [72]:
print(X_train1.shape)
print(Y_train1.shape)

(2703, 40)
(2703, 4)


In [23]:
"""
input_shape=X_train.shape
sentence_indices = Input(input_shape,dtype='int32')
vocab_len=len(words_set)+1
emb_dim=word_to_vec_map["cucumber"].shape[0]
#model = Sequential()
#model.add(Embedding(vocab_len,25))
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
# Propagate sentence_indices through your embedding layer
# (See additional hints in the instructions).
embeddings = embedding_layer(sentence_indices)    
#model.add(LSTM(128, dropout =0.3, recurrent_dropout=0.3,return_sequences=True))
X=LSTM(128, dropout =0.3, recurrent_dropout=0.3,return_sequences=True)(embeddings)
#model.add(LSTM(128))
X = LSTM(128, return_sequences=False)(X)
#model.add(Dense(4, activation='softmax'))
# try using different optimizers and different optimizer configs
#adam = optimizers.Adam(lr=0.1)
#model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
X = Dense(4)(X)
    # Add a softmax activation
X = Activation('softmax')(X)
model = Model(inputs=sentence_indices, outputs=X)
"""

'\ninput_shape=X_train.shape\nsentence_indices = Input(input_shape,dtype=\'int32\')\nvocab_len=len(words_set)+1\nemb_dim=word_to_vec_map["cucumber"].shape[0]\n#model = Sequential()\n#model.add(Embedding(vocab_len,25))\nembedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)\n# Propagate sentence_indices through your embedding layer\n# (See additional hints in the instructions).\nembeddings = embedding_layer(sentence_indices)    \n#model.add(LSTM(128, dropout =0.3, recurrent_dropout=0.3,return_sequences=True))\nX=LSTM(128, dropout =0.3, recurrent_dropout=0.3,return_sequences=True)(embeddings)\n#model.add(LSTM(128))\nX = LSTM(128, return_sequences=False)(X)\n#model.add(Dense(4, activation=\'softmax\'))\n# try using different optimizers and different optimizer configs\n#adam = optimizers.Adam(lr=0.1)\n#model.compile(loss=\'categorical_crossentropy\', optimizer=adam, metrics=[\'accuracy\'])\nX = Dense(4)(X)\n    # Add a softmax activation\nX = Activation(\'softmax\')(X

In [0]:
def LSTM_Model(input_shape, word_to_vec_map, word_to_index):
    
    sentence_indices = Input(input_shape,dtype='int32')
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    embeddings = embedding_layer(sentence_indices)  
    X = Bidirectional(LSTM(128, return_sequences=True))(embeddings)
    # dropout with a probability of 0.3
    X = Dropout(0.5)(X)
    #  X trough another BiLSTM layer with 128-dimensional hidden state
    X = Bidirectional(LSTM(128, return_sequences=False))(X)
    # dropout with a probability of 0.2
    X = Dropout(0.5)(X)
    #  X through a Dense layer with 5 units
    X = Dense(4)(X)
    # softmax activation
    X = Activation('softmax')(X)
    
    # Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices, outputs=X)
    
    return model

In [56]:
model1 = LSTM_Model((40,), word_to_vec_map, word_to_index)
model1.summary()


Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 40)                0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 40, 50)            20000100  
_________________________________________________________________
bidirectional_9 (Bidirection (None, 40, 256)           183296    
_________________________________________________________________
dropout_9 (Dropout)          (None, 40, 256)           0         
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 256)               394240    
_________________________________________________________________
dropout_10 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 1028

In [0]:
#adam = optimizers.Adam(lr=0.1)
checkpointer = ModelCheckpoint('best_weight_task2.hdf5', verbose=0, save_best_only=True)
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [74]:
model1.fit(X_train1, Y_train1, validation_data = [X_dev,Y_dev], callbacks=[checkpointer], shuffle=True, epochs = 40, batch_size = 32)

Train on 2703 samples, validate on 388 samples
Epoch 1/40
2703/2703 [==============================] - 25s 9ms/step - loss: 1.1558 - acc: 0.4880 - val_loss: 1.1062 - val_acc: 0.5412
Epoch 2/40
2703/2703 [==============================] - 21s 8ms/step - loss: 1.1098 - acc: 0.5165 - val_loss: 1.1215 - val_acc: 0.5284
Epoch 3/40
2703/2703 [==============================] - 21s 8ms/step - loss: 1.0703 - acc: 0.5353 - val_loss: 1.1225 - val_acc: 0.5180
Epoch 4/40
2703/2703 [==============================] - 21s 8ms/step - loss: 1.0210 - acc: 0.5542 - val_loss: 1.1712 - val_acc: 0.5155
Epoch 5/40
2703/2703 [==============================] - 21s 8ms/step - loss: 0.9678 - acc: 0.5768 - val_loss: 1.1693 - val_acc: 0.5103
Epoch 6/40
2703/2703 [==============================] - 21s 8ms/step - loss: 0.9410 - acc: 0.5934 - val_loss: 1.3588 - val_acc: 0.4407
Epoch 7/40
2703/2703 [==============================] - 21s 8ms/step - loss: 0.8721 - acc: 0.6286 - val_loss: 1.2356 - val_acc: 0.5180
Epoch 8/

In [0]:
model1.load_weights('best_weight_task2.hdf5')

In [80]:
loss, acc = model1.evaluate(X_dev, Y_dev)
print("Accuracy of the Classifier model on Dev data is:",acc*100)

388/388 [==============================] - 0s 1ms/step
Accuracy of the Classifier model on Dev data is: 54.123711340206185
